### Training data preparation

- Input: 
    1. Raw training images
    2. Instance segmentation maps from ilastik
   
- Applies: 
    1. Remove small objects
    2. Fill small holes
    3. Remove large objects
    4. Edge smoothing
    
- Output:
    1. .npz file (X = raw, instance = segmap, class = class labels) 
    

In [1]:
import numpy as np
import skimage as sk